![First Power Bi](.\images\dbachecks-logo.png )
# dbachecks - Saving Configurations

You can alter the configurations for any check at the command line to ensure that you are running checks that will test what you want.

You will want to save those configurations so that you can pass them to other people or run them using automation.

The process that you will need to follow is 

- Create the configuration
- test the configuration
- save the configuration with `Export-DbcConfig`

This will create a json file of the configuration which you can save in source control and then you, others in your team or automation can use the configuration with `Import-DbcConfig`

# Excluding and skipping checks

As well as the configuration items that you saw in the simple configuration notebook, which will change the value for a check such as making sure that the database owners are the accounts that you expect, there are other configuration items which enable you to exclude checks, instances or databases from checks, skip parts of checks.

You can see those configurations by running the block below.

When you are creating configurations for saving and sharing, you will find `command.invokedbccheck.excludecheck` and `command.invokedbccheck.excludedatabases` useful.

`command.invokedbccheck.excludecheck` will set the configuration so that any checks specified here will not run. It is best to exclude Groups of checks rather than individual ones here.
`command.invokedbccheck.excludedatabases` will set the configuration for any databases that you do not wish to run checks against.


In [1]:
Get-DbcConfig | Where-Object{$_.Name -like '*exclude*' -or $_.Name -like '*skip*'} 


Name                                                                   Value Description                               
----                                                                   ----- -----------                               
agent.failedjob.excludecancelled                                       False Exclude agent jobs with a status of can...
command.invokedbccheck.excludecheck                                       {} Invoke-DbcCheck: The checks that should...
command.invokedbccheck.excludedatabases                                   {} Invoke-DbcCheck: The databases that sho...
policy.adlogingroup.excludecheck                                             Active Directory Groups logins to exclu...
policy.adloginuser.excludecheck                                              Active Directory User logins to exclude...
policy.certificateexpiration.excludedb         {master, msdb, model, tempdb} Databases to exclude from expired certi...
policy.database.filegrowthexcludedb    

# Create a configuration

First you will need to create your configuration. 

## Reset the configuration to default first
It is best practice to run `Reset-DbcConfig` before you start to do this to avoid confusion. This will reset all of the configurations to the default values. It will also list all of the configurations and you can see the current (default) values  

In [1]:
Reset-DbcConfig


Name                                                                                                              Value
----                                                                                                              -----
agent.alert.Job                                                                                                   False
agent.alert.messageid                                                                                   {823, 824, 825}
agent.alert.Notification                                                                                           True
agent.alert.Severity                                                                                {16, 17, 18, 19...}
agent.databasemailprofile                                                                                              
agent.dbaoperatoremail                                                                                                 
agent.dbaoperatorname                  

## Morning Checks Configuration

Lets create a configuration for some morning checks for our Production estate. You will

- Check the production instances (we only have two in this demo) 'localhost,15592','localhost,15593'
- Ensure the instance is available and the connection authentication scheme is SQL (because we are using SQL Auth) but don't check PSRemoting
- Ensure that all agent jobs have succeeded
- All databases have backups that meet RPO/RTO
- There are no errors in the Error Logs in the last 24 hours
- The disk space is not less than 10% free
- There are no Memory Dumps
- The last duration of the agent jobs was not more than 10% of the average run time including any currently running jobs
- The log file sizes are less than 50% of the data file sizes
- That databases have more than 10% free space available


In [2]:
# Set the instances and computernames to run the check against
Set-DbcConfig -Name app.sqlinstance -Value 'localhost,15592','localhost,15593'
Set-DbcConfig -Name app.computername -Value 'localhost,15592','localhost,15593'
# Set the auth scheme to SQL as we are using SQL Auth
Set-DbcConfig -Name policy.connection.authscheme -Value SQL
# Don't check the PSRemoting
Set-DbcConfig -Name skip.connection.remoting -Value $true
# Set expected backup - 7 days for full - 24 hours for diff - 15 minutes for log
Set-DbcConfig -Name policy.backup.fullmaxdays -Value 7
Set-DbcConfig -Name policy.backup.diffmaxhours -Value 24
Set-DbcConfig -Name policy.backup.logmaxminutes -Value 15
# Set config for disk space
Set-DbcConfig -Name policy.diskspace.percentfree -Value 10
# Maximum number of memory dumps
Set-DbcConfig -Name policy.dump.maxcount -Value 1
# Set config for job run times
Set-DbcConfig -Name agent.lastjobruntime.percentage -Value 10
Set-DbcConfig -Name agent.longrunningjob.percentage -Value 10
# Set max size of log files compared to data file size
Set-DbcConfig -Name policy.database.logfilesizecomparison -Value Average
Set-DbcConfig -Name policy.database.logfilesizepercentage -Value 50
# Set config to ensure files have more than 10% free
Set-DbcConfig -Name policy.database.filegrowthfreespacethreshold -Value 10


Name                         Value                              Description                                            
----                         -----                              -----------                                            
app.sqlinstance              {localhost,15592, localhost,15593} List of SQL Server instances that SQL-based tests wi...
app.computername             {localhost,15592, localhost,15593} List of Windows Servers that Windows-based tests wil...
policy.connection.authscheme SQL                                Auth requirement (Kerberos, NTLM, etc)                 
skip.connection.remoting     True                               Skip PowerShell remoting check for connectivity        
policy.backup.fullmaxdays    7                                  Maximum number of days before Full Backups are consi...
policy.backup.diffmaxhours   24                                 Maximum number of hours before Diff Backups are cons...
policy.backup.logmaxminutes  15        

# Examining the current configuration and saving it

When you set the configuration, the output shows the cuurently configured values for the configuration items that you have changed.

You can examine the complete current configuration with `Get-DbcConfig`

In [3]:
Get-DbcConfig


Name                                                                                                              Value
----                                                                                                              -----
agent.alert.Job                                                                                                   False
agent.alert.messageid                                                                                   {823, 824, 825}
agent.alert.Notification                                                                                           True
agent.alert.Severity                                                                                {16, 17, 18, 19...}
agent.databasemailprofile                                                                                              
agent.dbaoperatoremail                                                                                                 
agent.dbaoperatorname                  

## Saving the configuration to a file for reuse

Once you have a configuration created, you can save it for easy importing into your session or for use with automation. I suggest that the best place for you to save this is in a source control repository so changes can be controlled and auditted

To save the current configuration use `Export-DbcConfig`

In [4]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
Export-DbcConfig -Path $FolderPath\Morning-Checks-Production.json 

[16:51:03][Export-DbcConfig] Wrote file to C:\Users\mrrob\Documents\dbachecks\Morning-Checks-Production.json


The file that is created is just json, you can take a look at it (and alter it directly if you wish)

In [6]:
azuredatastudio $FolderPath\Morning-Checks-Production.json 

# Loading the configuration from a file

To use the configuration that you have saved you use `Import-DbcConfig`

This will take the values from the json file and set the configuration accordingly

In [7]:
Import-DbcConfig -Path $FolderPath\Morning-Checks-Production.json 


Name                                              Value Description                                                    
----                                              ----- -----------                                                    
agent.alert.Job                                   False Agent alert job notification. Ex job to write to eventlog fo...
agent.alert.messageid                   {823, 824, 825} Agent alert messageid to validate; https://www.brentozar.com...
agent.alert.Notification                           True Agent alert notification                                       
agent.alert.Severity                {16, 17, 18, 19...} Agent alert severity to validate; https://www.brentozar.com/...
agent.databasemailprofile                               Name of the Database Mail Profile in SQL Agent                 
agent.dbaoperatoremail                                  Email address of the DBA Operator in SQL Agent                 
agent.dbaoperatorname                  

# Run the Checks with that configuration

You can then run the checks with that configuration as usual. You do not need to specify the computer names or Instance names as they are in the configuration already. You will reset the configuration, load the configuration from the file and run the checks

It will take about  30 seconds or so

In [8]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
# Just in case the config has been altered
$null = Reset-DbcConfig
$null = Import-DbcConfig -Path $FolderPath\Morning-Checks-Production.json 
$invokeDbcCheckSplat = @{

    SqlCredential = $SqlCredential
    Check = 'InstanceConnection', 'FailedJob', 'LastBackup', 'ErrorLog', 'DiskCapacity', 'MemoryDump', 'LastJobRunTime', 'LongRunningJob', 'LogfileSize', 'FutureFileGrowth'
}
Invoke-DbcCheck @invokeDbcCheckSplat

Executing all tests in 'C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.15\checks\Agent.Tests.ps1', 'C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.15\checks\Database.Tests.ps1', 'C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.15\checks\Instance.Tests.ps1', 'C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.15\checks\Server.Tests.ps1' with Tags InstanceConnection', 'FailedJob', 'LastBackup', 'ErrorLog', 'DiskCapacity', 'MemoryDump', 'LastJobRunTime', 'LongRunningJob', 'LogfileSize', 'FutureFileGrowth

Executing script C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.15\checks\Agent.Tests.ps1

  Describing Failed Jobs

    Context Checking for failed enabled jobs since 02/13/2020 16:00:25 on localhost,15592
      [+] CommandLog Cleanup's last run outcome on 0034794b4ce3 is Succeeded 368ms
      [-] DatabaseBackup - SYSTEM_DATABASES - FULL's last run outcome on 0034794b4ce3 is Failed 157ms
        Expected 'Succeeded', because All Agent Jobs sh

## Only showing the failures

This will run 91 checks and most will pass. To avoid filling our buffer will all the checks that have passed, we can only display the failed checks using the `-Show` parameter with the `Fails` value. (NOTE - This is a better option than `Failed` which will only show the failed tests and no context for them)

In [9]:
$invokeDbcCheckSplat = @{
    SqlCredential = $SqlCredential
    Check = 'InstanceConnection', 'FailedJob', 'LastBackup', 'ErrorLog', 'DiskCapacity', 'MemoryDump', 'LastJobRunTime', 'LongRunningJob', 'LogfileSize', 'FutureFileGrowth'
    Show = 'Fails'
}
Invoke-DbcCheck @invokeDbcCheckSplat

Executing all tests in 'C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.15\checks\Agent.Tests.ps1', 'C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.15\checks\Database.Tests.ps1', 'C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.15\checks\Instance.Tests.ps1', 'C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.15\checks\Server.Tests.ps1' with Tags InstanceConnection', 'FailedJob', 'LastBackup', 'ErrorLog', 'DiskCapacity', 'MemoryDump', 'LastJobRunTime', 'LongRunningJob', 'LogfileSize', 'FutureFileGrowth

Executing script C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.15\checks\Agent.Tests.ps1

  Describing Failed Jobs

    Context Checking for failed enabled jobs since 02/13/2020 16:04:48 on localhost,15592
      [-] DatabaseBackup - SYSTEM_DATABASES - FULL's last run outcome on 0034794b4ce3 is Failed 5ms
        Expected 'Succeeded', because All Agent Jobs should have succeed - you need to investigate the failed jobs, but got Failed.
        

# Configuration for New Instance

You might want to create a configuration to run after a new instance has been set up

This might require the following configurations (other settings will also be checked with default configurations)

- Database mail has been set up
- Operators and Failsafe Operators are set up
- Agent Jobs are owned by the correct account
- Agent Alerts are set up correctly
- Job History settings are set up correctly
- Ola Hallengrens Maintenance solution is set up as expected
- The expected Extended Events sessions are installed and running
- The correct Error Logs Setting



In [5]:
# reset the configuration
$null = Reset-DbcConfig
# Set Checks to exclude (these wont run on containers)
Set-DbcConfig -Name command.invokedbccheck.excludecheck -Value FailsafeOperator,DatabaseMailEnabled,DatabaseMailProfile,SqlEngineServiceAccount,OrphanedFile,ServerNameMatch
# Set the auth scheme to SQL as we are using SQL Auth
Set-DbcConfig -Name policy.connection.authscheme -Value SQL
# Don't check the PSRemoting
Set-DbcConfig -Name skip.connection.remoting -Value $true
# Set config for expected Operators
Set-DbcConfig -Name agent.dbaoperatorname -Value SQLAdmins
# Set COnfig for operators email address
Set-DbcConfig -Name agent.dbaoperatoremail -Value SQLAdmins@TheBeard.local
# Set config for the expected job owner
Set-DbcConfig -Name agent.validjobowner.name -Value OldSa
# Set config for expected agent alerts
$Ids = '1101','1105','1121','1214','17130','17179','17300','17883','17884','17887','17888','17890','2508','2511','28036','3271','3452','3619','3624','5180','5228','5229','5242','5243','5250','5572','5901','605','701','802','823','824','825','832','833','845','855','856','8966','9002','9100'
Set-DbcConfig -Name agent.alert.messageid -Value $Ids
# Set the config for the job history settings
Set-DbcConfig -Name agent.history.maximumhistoryrows -Value 1000
Set-DbcConfig -Name agent.history.maximumjobhistoryrows -Value 100
# Set config for the extended events
Set-DbcConfig -Name policy.xevent.requiredrunningsession -Value '15 Second IO Error','Blocked Process Report','Stored Procedure Parameters' 
# Sets Error log - Its a container!
Set-DbcConfig -Name policy.errorlog.logcount -Value -1
# Set Remote Access configuration
Set-DbcConfig -Name skip.instance.remoteaccessdisabled -Value $true




Name                                Value                                                                              
----                                -----                                                                              
command.invokedbccheck.excludecheck {FailsafeOperator, DatabaseMailEnabled, DatabaseMailProfile, SqlEngineServiceAcc...
policy.connection.authscheme        SQL                                                                                
skip.connection.remoting            True                                                                               
agent.dbaoperatorname               SQLAdmins                                                                          
agent.dbaoperatoremail              SQLAdmins@TheBeard.local                                                           
agent.validjobowner.name            OldSa                                                                              
agent.alert.messageid               {11

With the configuration created you can save it to a file

In [6]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
Export-DbcConfig -Path $FolderPath\New-Instance.json 

[16:51:48][Export-DbcConfig] Wrote file to C:\Users\mrrob\Documents\dbachecks\New-Instance.json


You can then load that configuration and run it. This will take a couple of minutes to run, be patient.

In [13]:
# Just in case the config has been altered
$null = Reset-DbcConfig
$null = Import-DbcConfig -Path $FolderPath\New-Instance.json 
$invokeDbcCheckSplat = @{
    SqlInstance = 'localhost,15592'
    SqlCredential = $SqlCredential
    Check = 'Instance','Agent'
    Show = 'Fails'
}
Invoke-DbcCheck @invokeDbcCheckSplat

Executing all tests in 'C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.15\checks\Agent.Tests.ps1', 'C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.15\checks\Instance.Tests.ps1' with Tags Instance', 'Agent

Executing script C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.15\checks\Agent.Tests.ps1

  Describing SQL Agent Account
    [!] Running on Linux or connecting to container so can't check Services on localhost,15592, is skipped 0ms

  Describing DBA Operators

    Context Testing DBA Operators exists on localhost,15592

  Describing Failed Jobs

    Context Checking for failed enabled jobs since 02/13/2020 16:06:43 on localhost,15592
      [-] DatabaseBackup - SYSTEM_DATABASES - FULL's last run outcome on 0034794b4ce3 is Failed 5ms
        Expected 'Succeeded', because All Agent Jobs should have succeed - you need to investigate the failed jobs, but got Failed.
        170:                                         $psitem.LastRunOutcome | Should -Be "Suc

# Configuration for Weekly Check

This configuration could be used for a weekly check of an estate. It includes

- Failed jobs in the last 2 days - I ran this on a Sunday so checking weekend maintenance



In [7]:
# reset the configuration
$null = Reset-DbcConfig
# Set Checks to exclude (these wont run on containers)
Set-DbcConfig -Name command.invokedbccheck.excludecheck -Value FailsafeOperator,DatabaseMailEnabled,DatabaseMailProfile,SqlEngineServiceAccount,OrphanedFile,ServerNameMatch, SqlBrowserServiceAccount
# Set the instances and computernames to run the check against
Set-DbcConfig -Name app.sqlinstance -Value 'localhost,15592','localhost,15593'
Set-DbcConfig -Name app.computername -Value 'localhost,15592','localhost,15593'
# Set the auth scheme to SQL as we are using SQL Auth
Set-DbcConfig -Name policy.connection.authscheme -Value SQL
# Don't check the PSRemoting
Set-DbcConfig -Name skip.connection.remoting -Value $true
Set-DbcConfig -Name agent.failedjob.since -Value 2
# COnfig for build warning
Set-DbcConfig -Name policy.build.behind	-Value 1SP
Set-DbcConfig -Name policy.build.warningwindow	-Value 2
# COnfig for when we want to know certs are going to expire
Set-DbcConfig -Name policy.certificateexpiration.warningwindow -Value 2	
# How different can our files be in a filegroup
Set-DbcConfig -Name policy.database.filebalancetolerance -Value 3
# Free Space in file
Set-DbcConfig -Name policy.database.filegrowthfreespacethreshold 10
# How big can our log file be in comparison to our data files
Set-DbcConfig -Name policy.database.logfilesizecomparison -Value average	
Set-DbcConfig -Name policy.database.logfilesizepercentage -Value 50
# max number of VLFs allowed
Set-DbcConfig -Name policy.database.maxvlf 256
# Last DBCC
Set-DbcConfig -Name policy.dbcc.maxdays -Value 3
#Disk Space
Set-DbcConfig -Name policy.diskspace.percentfree -Value 5
# Set config for the expected job owner
Set-DbcConfig -Name agent.validjobowner.name -Value OldSa
# Set config for expected agent alerts
$Ids = '1101','1105','1121','1214','17130','17179','17300','17883','17884','17887','17888','17890','2508','2511','28036','3271','3452','3619','3624','5180','5228','5229','5242','5243','5250','5572','5901','605','701','802','823','824','825','832','833','845','855','856','8966','9002','9100'
Set-DbcConfig -Name agent.alert.messageid -Value $Ids
# Set the config for the job history settings
Set-DbcConfig -Name agent.history.maximumhistoryrows -Value 1000
Set-DbcConfig -Name agent.history.maximumjobhistoryrows -Value 100
# Set config for the extended events
Set-DbcConfig -Name policy.xevent.requiredrunningsession -Value '15 Second IO Error','Blocked Process Report','Stored Procedure Parameters' 
# Set Database owners
Set-DbcConfig -Name policy.invaliddbowner.name -Value OldSa
Set-DbcConfig -Name policy.validdbowner.name -Value sqladmin

Set-DbcConfig -Name database.exists -Value 'pubs','NorthWind','AdventureWorks2017' -Append



Name                                Value                                                                              
----                                -----                                                                              
command.invokedbccheck.excludecheck {FailsafeOperator, DatabaseMailEnabled, DatabaseMailProfile, SqlEngineServiceAcc...
app.sqlinstance                     {localhost,15592, localhost,15593}                                                 
app.computername                    {localhost,15592, localhost,15593}                                                 
policy.connection.authscheme        SQL                                                                                
skip.connection.remoting            True                                                                               
agent.failedjob.since               2                                                                                  
policy.build.behind                 1SP

and save it

In [8]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
Export-DbcConfig -Path $FolderPath\Weekly-Check.json 

[16:52:11][Export-DbcConfig] Wrote file to C:\Users\mrrob\Documents\dbachecks\Weekly-Check.json


load it and run it.  

This will run nearly 900 tests and takes a minute or so

In [16]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
# Just in case the config has been altered
$null = Reset-DbcConfig
$null = Import-DbcConfig -Path $FolderPath\Weekly-Check.json 
$invokeDbcCheckSplat = @{
    SqlInstance = 'localhost,15592'
    SqlCredential = $SqlCredential
    Check = 'Agent', 'Database','Instance'
    Show = 'Fails'
}
Invoke-DbcCheck @invokeDbcCheckSplat

Executing all tests in 'C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.15\checks\Agent.Tests.ps1', 'C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.15\checks\Database.Tests.ps1', 'C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.15\checks\Instance.Tests.ps1' with Tags Agent', 'Database', 'Instance

Executing script C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.15\checks\Agent.Tests.ps1

  Describing SQL Agent Account
    [!] Running on Linux or connecting to container so can't check Services on localhost,15592, is skipped 0ms

  Describing DBA Operators

    Context Testing DBA Operators exists on localhost,15592

  Describing Failed Jobs

    Context Checking for failed enabled jobs since 03/12/2020 16:08:07 on localhost,15592
      [-] DatabaseBackup - SYSTEM_DATABASES - FULL's last run outcome on 0034794b4ce3 is Failed 4ms
        Expected 'Succeeded', because All Agent Jobs should have succeed - you need to investigate the failed jobs, but got 